<a href="https://colab.research.google.com/github/francescogrillea/AnomalyDetectionTFLite/blob/main/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import e Costanti

In [4]:
!cat ./drive/MyDrive/Colab*/anomaly_detection_data/res*

CPU
	Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz

RAM
	10G / 13G free
Thu Dec  2 18:46:11 2021
Architecture: LSTM_1L
Channel: A-1
Found untraced functions such as lstm_cell_layer_call_fn, lstm_cell_layer_call_and_return_conditional_losses, lstm_cell_layer_call_fn, lstm_cell_layer_call_and_return_conditional_losses, lstm_cell_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
Assets written to: /tmp/tmpx6xzzm9q/assets
Buffer deduplication procedure will be skipped when flatbuffer library is not properly loaded
Model A-1.h5 converted (from 47Kb to 15Kb)
TensorFlow prediction time: 0.7093887329101562s
TensorFlowLite prediction time: 0.0002696514129638672s
TensorFlow predictions	TFLite predictions	Distance
0.9981911182403564	0.9926959276199341	0.005495190620422363
0.9989597797393799	0.9959942698478699	0.0029655098915100098
0.9989480376243591	0.9968134760856628	0.002134561538696289
0.9989720582962036	0

In [2]:
!rm ./drive/MyDrive/Colab*/anomaly_detection_data/res*

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
import logging
import json

cpu = os.popen('lscpu |grep \'Model name\'').read()
ram_free = os.popen('free -h --si | awk  \'/Mem:/{print $4}\'').read()
ram_total = os.popen('free -h --si | awk  \'/Mem:/{print $2}\'').read()

logging.basicConfig(filename="./drive/MyDrive/Colab Notebooks/anomaly_detection_data/results.log", level=logging.INFO, format='%(message)s')
logging.info('CPU\n\t{}'.format(cpu))
logging.info('RAM\n\t{} / {} free'.format(ram_free[:-1], ram_total[:-1]))

results = {}

In [15]:
PROJECT_NAME = 'anomaly_detection'
PROJECT_PATH = './drive/MyDrive/Colab Notebooks/anomaly_detection_data/'
DATASET_PATH = PROJECT_PATH+'Dataset/'
ARCHITECTURE_TYPE = 'LSTM_1L/'
TF_MODELS_PATH = PROJECT_PATH+'TF_Models/'+ARCHITECTURE_TYPE
TFLite_MODELS_PATH = PROJECT_PATH+'TFLite_Models/'+ARCHITECTURE_TYPE
CHANNEL_NAME = 'A-3'

results[CHANNEL_NAME] = {}
results[CHANNEL_NAME]['timestamp'] = time.ctime()
results[CHANNEL_NAME]['architecture'] = ARCHITECTURE_TYPE[:-1]

logging.info('---------------------------------')
logging.info('{}\nArchitecture: {}\nChannel: {}'.format(time.ctime(), ARCHITECTURE_TYPE[:-1], CHANNEL_NAME))

# Load Model and Dataset

In [2]:
def load_tf_model(channel_name):
  if not channel_name.endswith('.h5'):
    channel_name = channel_name+'.h5'
  tf_model = keras.models.load_model(TF_MODELS_PATH+channel_name)
  return tf_model


def load_dataset(channel_name):
  if not channel_name.endswith('.npy'):
    channel_name = channel_name+'.npy'
  x = np.array([np.load(DATASET_PATH+'test/'+channel_name)])
  y = np.array([])
  return x,y

# Prediction su TF Model

In [18]:
#load dataset
tf_input_data, tf_output_data = load_dataset(CHANNEL_NAME)

model = load_tf_model(CHANNEL_NAME)
start_time = time.time()
tf_output_data = model.predict(tf_input_data)
delta_time = time.time()-start_time

results[CHANNEL_NAME]['tf_prediction_time'] = delta_time
results[CHANNEL_NAME]['tf_predictions'] = tf_output_data
logging.info('TensorFlow prediction time: {}s'.format(delta_time))
print('Library: {}\nChannel: {}\nPredictions: {}\nTime elapsed: {}s'.format('TensorFlow', CHANNEL_NAME, tf_output_data, delta_time))

Library: TensorFlow
Channel: A-3
Predictions: [[0.38492486 0.42751762 0.4466028  0.4841723  0.53786206 0.56776667
  0.6096827  0.61953366 0.64891905 0.6596439 ]]
Time elapsed: 0.6823575496673584s


# Conversione dei modelli da .h5 a .tflite

In [17]:
channels = [f for f in os.listdir(TF_MODELS_PATH)]

for model_filename in channels:
  if not model_filename.startswith(CHANNEL_NAME):
    continue

  #in Kb
  tf_model_size = int(os.path.getsize(TF_MODELS_PATH+model_filename) / 1024)
  results[CHANNEL_NAME]['tf_model_size'] = tf_model_size

  tf_model = load_tf_model(model_filename)
  converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
  converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  results[CHANNEL_NAME]['optimizations'] = 'DEFAULT'
  #WARNING:absl:Found untraced functions such as lstm_cell -> e' un warning che dipende da come è stato salvato il modello, tuttavia non sembra esserci una precisa soluzione
  tflite_model = converter.convert()

  #generate TFLite model name
  channel_name = model_filename[:-3]
  tflite_model_filename = channel_name+'.tflite'
  
  TFLite_MODEL_FILE = TFLite_MODELS_PATH+tflite_model_filename
  with open(TFLite_MODEL_FILE, 'wb') as f:
    f.write(tflite_model)
    print('Model for channel '+channel_name+' converted')
    
  #in Kb  
  tflite_model_size = int(os.path.getsize(TFLite_MODEL_FILE) / 1024)
  results[CHANNEL_NAME]['tflite_model_size'] = tflite_model_size
  logging.info('Model '+model_filename+' converted (from '+str(tf_model_size)+'Kb to '+str(tflite_model_size)+'Kb)')

INFO:tensorflow:Assets written to: /tmp/tmpmzd8wxbg/assets
Model for channel A-3 converted


# Prediction su TFLite

In [19]:
#load dataset
x_data, tflite_output_data = load_dataset(CHANNEL_NAME)

interpreter = tf.lite.Interpreter(model_path=TFLite_MODELS_PATH+CHANNEL_NAME+'.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#preprocessing input data
input_shape = input_details[0]['shape']

start_time = time.time()
#per ogni riga del dataset, creo un tensore e invoco il modello
for data in x_data[0]:
  input_data = np.array([[data]], dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
    
  #run inference
  start_time = time.time()
  interpreter.invoke()
  stop_time = time.time()
  
  #recupero l'output prodotto dal modello
  output_data = interpreter.get_tensor(output_details[0]['index'])
  tflite_output_data = np.append(tflite_output_data, output_data)

delta_time = time.time() - start_time
results[CHANNEL_NAME]['tflite_prediction_time'] = delta_time
results[CHANNEL_NAME]['tflite_predictions'] = tflite_output_data
logging.info('TensorFlowLite prediction time: {}s'.format(delta_time))
print('Library: {}\nChannel: {}\nPredictions: {}\nTime elapsed: {}s'.format('TensorFlow Lite', CHANNEL_NAME, tflite_output_data, delta_time))

Library: TensorFlow Lite
Channel: A-3
Predictions: [0.07142104 0.07733905 0.06671091 ... 0.12977967 0.1367701  0.13243306]
Time elapsed: 0.0004017353057861328s


In [20]:
#confronto tra i risultati prodotti dal modello TF (sinistra) rispetto al modello TFLite (destra)
logging.info('TensorFlow predictions\tTFLite predictions\tDistance')
results[CHANNEL_NAME]['distance'] = []
for y_tf, y_tflite in zip(tf_output_data[0], tflite_output_data):
  logging.info('{}\t{}\t{}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))
  print('{} {} {}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))
  results[CHANNEL_NAME]['distance'].append(abs(y_tf-y_tflite))

logging.info('-----------------------------------------')

0.38492485880851746 0.07142104208469391 0.31350381672382355
0.4275176227092743 0.0773390531539917 0.3501785695552826
0.44660279154777527 0.06671091169118881 0.37989187985658646
0.4841723144054413 0.0747588723897934 0.4094134420156479
0.5378620624542236 0.0795966237783432 0.45826543867588043
0.5677666664123535 0.07393834739923477 0.49382831901311874
0.6096826791763306 0.07659672200679779 0.5330859571695328
0.6195336580276489 0.08126199245452881 0.5382716655731201
0.6489190459251404 0.076280876994133 0.5726381689310074
0.6596438884735107 0.07287921011447906 0.5867646783590317


In [21]:
print(results)

{'A-1': {'timestamp': 'Thu Dec  2 18:30:56 2021', 'architecture': 'LSTM_1L', 'tf_model_size': 47, 'optimizations': 'DEFAULT', 'tflite_model_size': 15}, 'A-2': {'timestamp': 'Thu Dec  2 18:34:33 2021', 'architecture': 'LSTM_1L', 'tf_model_size': 330, 'optimizations': 'DEFAULT', 'tflite_model_size': 38, 'tf_prediction_time': 0.7515320777893066, 'tf_predictions': array([[-0.05996562, -0.03316002, -0.00884144, -0.01433018, -0.05385279,
        -0.11776038, -0.16840929, -0.2367448 , -0.35468954, -0.47840437]],
      dtype=float32), 'tflite_prediction_time': 0.0003933906555175781, 'tflite_predictions': array([-0.09370165, -0.08716335, -0.09235501, ..., -0.02809398,
       -0.03174705, -0.03838697]), 'distance': [0.033736035227775574, 0.054003335535526276, 0.08351356722414494, 0.07393042929470539, 0.031069528311491013, 0.023678027093410492, 0.08553766459226608, 0.18102655559778214, 0.2562684640288353, 0.38439059257507324]}, 'A-3': {'timestamp': 'Thu Dec  2 18:35:26 2021', 'architecture': 'LST

## Completo per Loop

In [3]:
PROJECT_NAME = 'anomaly_detection'
PROJECT_PATH = './drive/MyDrive/Colab Notebooks/anomaly_detection_data/'
DATASET_PATH = PROJECT_PATH+'Dataset/'
ARCHITECTURE_TYPE = 'LSTM_1L/'
TF_MODELS_PATH = PROJECT_PATH+'TF_Models/'+ARCHITECTURE_TYPE
TFLite_MODELS_PATH = PROJECT_PATH+'TFLite_Models/'+ARCHITECTURE_TYPE
CHANNEL_NAME = ['A-1','A-2','A-3','A-4','A-5','A-6','A-7','A-8','A-9']

for i in range(9):
  logging.info('{}\nArchitecture: {}\nChannel: {}'.format(time.ctime(), ARCHITECTURE_TYPE[:-1], CHANNEL_NAME[i]))
  channels = [f for f in os.listdir(TF_MODELS_PATH)]

  for model_filename in channels:
    if not model_filename.startswith(CHANNEL_NAME[i]):
      continue

    #in Kb
    tf_model_size = int(os.path.getsize(TF_MODELS_PATH+model_filename) / 1024)
    #results[CHANNEL_NAME]['tf_model_size'] = tf_model_size

    tf_model = load_tf_model(model_filename)
    converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
    converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    #results[CHANNEL_NAME]['optimizations'] = 'DEFAULT'
    #WARNING:absl:Found untraced functions such as lstm_cell -> e' un warning che dipende da come è stato salvato il modello, tuttavia non sembra esserci una precisa soluzione
    tflite_model = converter.convert()

    #generate TFLite model name
    channel_name = model_filename[:-3]
    tflite_model_filename = channel_name+'.tflite'
    
    TFLite_MODEL_FILE = TFLite_MODELS_PATH+tflite_model_filename
    with open(TFLite_MODEL_FILE, 'wb') as f:
      f.write(tflite_model)
      print('Model for channel '+channel_name+' converted')
      
    #in Kb  
    tflite_model_size = int(os.path.getsize(TFLite_MODEL_FILE) / 1024)
    #results[CHANNEL_NAME]['tflite_model_size'] = tflite_model_size
    logging.info('Model '+model_filename+' converted (from '+str(tf_model_size)+'Kb to '+str(tflite_model_size)+'Kb)')

  #PREDICTION on TF

  #load dataset
  tf_input_data, tf_output_data = load_dataset(CHANNEL_NAME[i])

  model = load_tf_model(CHANNEL_NAME[i])
  start_time = time.time()
  tf_output_data = model.predict(tf_input_data)
  delta_time = time.time()-start_time

  logging.info('TensorFlow prediction time: {}s'.format(delta_time))
  print('Library: {}\nChannel: {}\nPredictions: {}\nTime elapsed: {}s'.format('TensorFlow', CHANNEL_NAME[i], tf_output_data, delta_time))

  #PREDICTION ON TFLITE

  #load dataset
  x_data, tflite_output_data = load_dataset(CHANNEL_NAME[i])

  interpreter = tf.lite.Interpreter(model_path=TFLite_MODELS_PATH+CHANNEL_NAME[i]+'.tflite')
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  #preprocessing input data
  input_shape = input_details[0]['shape']

  start_time = time.time()
  #per ogni riga del dataset, creo un tensore e invoco il modello
  for data in x_data[0]:
    input_data = np.array([[data]], dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
      
    #run inference
    start_time = time.time()
    interpreter.invoke()
    stop_time = time.time()
    
    #recupero l'output prodotto dal modello
    output_data = interpreter.get_tensor(output_details[0]['index'])
    tflite_output_data = np.append(tflite_output_data, output_data)

  delta_time = time.time() - start_time
  logging.info('TensorFlowLite prediction time: {}s'.format(delta_time))
  print('Library: {}\nChannel: {}\nPredictions: {}\nTime elapsed: {}s'.format('TensorFlow Lite', CHANNEL_NAME[i], tflite_output_data, delta_time))

  #difference 

  logging.info('TensorFlow predictions\tTFLite predictions\tDistance')
  for y_tf, y_tflite in zip(tf_output_data[0], tflite_output_data):
    logging.info('{}\t{}\t{}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))
    print('{} {} {}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))

INFO:tensorflow:Assets written to: /tmp/tmpx6xzzm9q/assets
Model for channel A-1 converted
Library: TensorFlow
Channel: A-1
Predictions: [[0.9981911  0.9989598  0.99894804 0.99897206 0.9992333  0.99905074
  0.9991313  0.99830365 0.9991017  0.99896634]]
Time elapsed: 0.7093887329101562s
Library: TensorFlow Lite
Channel: A-1
Predictions: [0.99269593 0.99599427 0.99681348 ... 0.99395204 0.99885583 0.99699628]
Time elapsed: 0.0002696514129638672s
0.9981911182403564 0.9926959276199341 0.005495190620422363
0.9989597797393799 0.9959942698478699 0.0029655098915100098
0.9989480376243591 0.9968134760856628 0.002134561538696289
0.9989720582962036 0.9970735907554626 0.0018984675407409668
0.9992333054542542 0.9978994131088257 0.0013338923454284668
0.9990507364273071 0.9971656203269958 0.0018851161003112793
0.9991313219070435 0.9977410435676575 0.0013902783393859863
0.9983036518096924 0.9939520359039307 0.004351615905761719
0.9991016983985901 0.9988558292388916 0.00024586915969848633
0.9989663362503